In [1]:
%matplotlib inline

In [2]:
device = 'cuda:0'

dataset_name = 'CIFAR10'
data_root = './data'

epochs = 100

In [3]:
from torchvision import models

model = models.ResNet(models.resnet.Bottleneck, [3, 4, 6, 3])

In [4]:
from torchvision import datasets

dataset = getattr(datasets, dataset_name)(root=data_root, train=True, download=True)

Files already downloaded and verified


In [5]:
dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train

In [6]:
dataset.class_to_idx

{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

In [7]:
import numpy as np
import pandas as pd

target_classes = ['frog', 'horse']
idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}

class_labels = pd.Series(dataset.targets).map(idx_to_class)
interested_idx = np.where(class_labels.isin(target_classes))[0]
subset_labels = class_labels.loc[interested_idx]

label_map = {dataset.class_to_idx[cls]: ix for ix, cls in enumerate(target_classes)}
target_transform = lambda x: label_map[x]

del dataset

In [8]:
from sklearn.model_selection import train_test_split

train_idx, val_idx = train_test_split(interested_idx, test_size=0.1, stratify=subset_labels)

In [11]:
from torch.utils.data import DataLoader, SubsetRandomSampler
import torchvision.transforms as tvt

train_xform = tvt.Compose([
    tvt.RandomAffine(degrees=20, shear=0.1),
    tvt.RandomHorizontalFlip(p=0.5),
    tvt.ToTensor()
])
val_xform = tvt.ToTensor()

train_dataset = getattr(datasets, dataset_name)(root=data_root, train=True, transform=train_xform, target_transform=target_transform)
val_dataset = getattr(datasets, dataset_name)(root=data_root, train=True, transform=val_xform, target_transform=target_transform)

train_dl = DataLoader(train_dataset, batch_size=256, sampler=SubsetRandomSampler(train_idx), pin_memory=True, num_workers=2)
val_dl = DataLoader(val_dataset, batch_size=256, sampler=SubsetRandomSampler(val_idx), pin_memory=True, num_workers=2)

In [1]:
%%capture
model

NameError: name 'model' is not defined

In [13]:
from torch import nn

model.fc = nn.Linear(model.fc.in_features, len(target_classes))
nn.init.kaiming_normal_(model.fc.weight)
nn.init.constant_(model.fc.bias, 0.1)

model.fc = nn.Sequential(nn.Dropout(0.5), model.fc)

In [14]:
%%capture
model.to(device)

In [13]:
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau


criterion = CrossEntropyLoss(reduction='mean').to(device)
optimizer = SGD(params=model.parameters(), lr=0.01, weight_decay=5e-5)
scheduler = ReduceLROnPlateau(optimizer, patience=10, cooldown=6, factor=0.1, verbose=True)

In [15]:
import torch
from collections import defaultdict
from tqdm import tqdm
import json

In [14]:
from inferno.utils.train_utils import AverageMeter
from sklearn.metrics import classification_report

best_loss = np.inf
for epoch in range(epochs):
    print(f'epoch: {epoch}')
    
    model.train()
    train_trackers = defaultdict(AverageMeter)
    all_labels, all_preds = [], []
    for image, label in train_dl:
        images = image.to(device)
        labels = label.to(device)
        
        logits = model(images)
        loss = criterion(logits, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        all_labels.extend(label.numpy())
        all_preds.extend(logits.argmax(dim=1).detach().cpu().numpy().tolist())
        train_trackers['loss'].update(loss.item())
        
    print('train: {}'.format({k: v.avg for k, v in train_trackers.items()}))
    print(f'train: {classification_report(y_true=all_labels, y_pred=all_preds)}')
        
    model.eval()
    val_trackers = defaultdict(AverageMeter)
    all_labels, all_preds = [], []
    for image, label in val_dl:
        images = image.to(device)
        labels = label.to(device)
        
        with torch.no_grad():
            logits = model(images)
            
        loss = criterion(logits, labels)
        
        all_labels.extend(label.numpy())
        all_preds.extend(logits.argmax(dim=1).detach().cpu().numpy().tolist())
        val_trackers['loss'].update(loss.item())
    
    print('val: {}'.format({k: v.avg for k, v in val_trackers.items()}))
    print(f'val: {classification_report(y_true=all_labels, y_pred=all_preds)}')
    
    val_loss = val_trackers['loss'].avg
    scheduler.step(val_loss)
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), 'best.pth')
        

epoch: 0
train: {'loss': 1.6810294439395268}
train:               precision    recall  f1-score   support

           0       0.60      0.62      0.61      4500
           1       0.61      0.60      0.60      4500

    accuracy                           0.61      9000
   macro avg       0.61      0.61      0.61      9000
weighted avg       0.61      0.61      0.61      9000

val: {'loss': 5.310507297515869}
val:               precision    recall  f1-score   support

           0       0.54      0.96      0.69       500
           1       0.82      0.18      0.30       500

    accuracy                           0.57      1000
   macro avg       0.68      0.57      0.49      1000
weighted avg       0.68      0.57      0.49      1000

epoch: 1
train: {'loss': 0.9385262570447392}
train:               precision    recall  f1-score   support

           0       0.68      0.70      0.69      4500
           1       0.69      0.67      0.68      4500

    accuracy                           0

In [16]:
print(len(train_idx))
print(len(val_idx))

9000
1000


In [17]:
%%capture
model.load_state_dict(torch.load('best.pth'), strict=True)
model.eval()

In [18]:
dataset = getattr(datasets, dataset_name)(root=data_root, train=False)
class_labels = pd.Series(dataset.targets).map(idx_to_class)
test_id_idx = np.where(class_labels.isin(np.unique(subset_labels)))[0]
test_ood_idx = np.where(~class_labels.isin(np.unique(subset_labels)))[0]

del dataset

In [19]:
print(len(test_id_idx))
print(len(test_ood_idx))

2000
8000


In [20]:
test_id_dataset = getattr(datasets, dataset_name)(root=data_root, train=False, transform=val_xform, target_transform=target_transform)
test_ood_dataset = getattr(datasets, dataset_name)(root=data_root, train=False, transform=val_xform)

test_id_dl = DataLoader(test_id_dataset, batch_size=1, sampler=SubsetRandomSampler(test_id_idx), pin_memory=True, num_workers=2)
test_ood_dl = DataLoader(test_ood_dataset, batch_size=1, sampler=SubsetRandomSampler(test_ood_idx), pin_memory=True, num_workers=2)

In [47]:
from captum.attr import LayerConductance
layers = ['layer1', 'layer2', 'layer3', 'layer4']

lcs = {l: LayerConductance(model, getattr(model, l)) for l in layers}

In [48]:
torch.cuda.empty_cache()

In [61]:
import os
import numpy as np
from torch.nn import functional as F

data_loader = DataLoader(val_dataset, batch_size=1, sampler=SubsetRandomSampler(val_idx), pin_memory=True, num_workers=2)
loader_name = 'val-id'

for layer_name, lc in lcs.items():
    attribution_matrix = None
    val_labels = np.zeros((len(val_idx)), dtype='uint8')
    print(f'### {layer_name}')
    for ix, (images, labels) in enumerate(data_loader):
        attribution = lc.attribute(images.to(device), target=labels[0])
        pooled_attribution = F.adaptive(attribution, 1).cpu().detach().numpy().squeeze().astype('float16')
        try:
            attribution_matrix[ix, :] = pooled_attribution
        except TypeError:
            attribution_matrix = np.zeros((len(val_idx), len(pooled_attribution)), dtype='float16')
            attribution_matrix[ix, :] = pooled_attribution
        val_labels[ix] = labels.item()
            
    print(attribution_matrix.shape)
    np.save(os.path.join('out', f'{loader_name}_{layer_name}.npy'), attribution_matrix)
    np.save(os.path.join('out', f'{loader_name}_{layer_name}_labels.npy'), val_labels)
    
np.save(os.path.join('out', 'val-id_index.npy'), val_idx)

### layer1
(1000, 256)
### layer2
(1000, 512)
### layer3
(1000, 1024)
### layer4
(1000, 2048)


In [64]:
import os
import numpy as np
from torch.nn import functional as F

targets = [0, 1]
for loader_name, data_loader, indices in zip(['test-id', 'test-ood'], [test_id_dl, test_ood_dl], [test_id_idx, test_ood_idx]):
    print(f'# {loader_name}')
    for layer_name, lc in lcs.items():
        attribution_matrix = None
        test_labels = np.zeros((len(indices)), dtype='uint8')
        print(f'### {layer_name}')
        for im_ix, (images, labels) in enumerate(data_loader):
            for target in targets:
                attribution = lc.attribute(images.to(device), target=target)
                pooled_attribution = F.adaptive_avg_pool2d(attribution, 1).cpu().detach().numpy().squeeze().astype('float16')
                try:
                    attribution_matrix[target, im_ix, :] = pooled_attribution
                except TypeError:
                    attribution_matrix = np.zeros((len(targets), len(indices), len(pooled_attribution)), dtype='float16')
                    attribution_matrix[target, im_ix, :] = pooled_attribution
            test_labels[im_ix] = labels.item()

        print(attribution_matrix.shape)
        np.save(os.path.join('out', f'{loader_name}_{layer_name}.npy'), attribution_matrix)
        np.save(os.path.join('out', f'{loader_name}_{layer_name}_labels.npy'), test_labels)
    np.save(os.path.join('out', f'{loader_name}_index.npy'), indices)

# test-id
### layer1
(2, 2000, 256)
### layer2
(2, 2000, 512)
### layer3
(2, 2000, 1024)
### layer4
(2, 2000, 2048)
# test-ood
### layer1
(2, 8000, 256)
### layer2
(2, 8000, 512)
### layer3
(2, 8000, 1024)
### layer4
(2, 8000, 2048)


In [46]:
attribution.size()

torch.Size([1, 2048, 1, 1])